In [90]:
from qiskit import QuantumProgram, QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random, pickle, time, datetime, xlsxwriter
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)
import operator

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

backend = "local_qasm_simulator"




Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\qecc_benchmark\Qconfig.py.

Available backends:  ['ibmqx5', 'local_qasm_simulator', 'ibmqx2', 'local_clifford_simulator', 'ibmq_qasm_simulator', 'local_statevector_simulator', 'ibmqx4', 'local_unitary_simulator']


In [105]:
# Grabs quantum register k from qc
def getQReg(qc,k):   
    desired = list(qc.get_qregs().items())
    return desired[k][1]

# Grabs classical register k from qc
def getCReg(qc,k):   
    desired = list(qc.get_cregs().items())
    return desired[k][1]  

class FiveOneThree:
    # Number of qubits
    n = 5
    
    # Defines code space
    validOutputs = ["00000", "10010", "01001", "10100", "01010", "11011", "00110",
                    "11000", "11101", "00011", "11110", "01111", "10001", "01100",
                    "10111", "00101", "11111", "01101", "10110", "01011", "10101",
                    "00100", "11001", "00111", "00010", "11100", "00001", "10000",
                    "01110", "10011", "01000", "11010"]

    # Read things left to right
    ind = {6:1, 7:0}
    
    # Sets mapping of qubit numbering so that qubits numbered "1 to n" appear
    # in that order left to right
    perm = {}
    
    def setPerm(self):
        n = self.n
        for i in range(n):
            self.perm[n-i] = i

    
    # prepare a state of all zeros
    def prepareZeros(self,qc):
        #FILL ME IN
        # First time we add error correction
        # 2 ancilla qubits used to check
        qr = getQReg(qc,0)
        
        anc = QuantumRegister(2)
        reg = ClassicalRegister(2)
        qc.add(anc)
        qc.add(reg)
        
        correct = False
        
        while not correct:
            # Actual State Preparation
            # Start by preparing +
            qc.h(qr)
            qc.cz(qr[perm[1]],qr[perm[2]])
            qc.cz(qr[perm[3]],qr[perm[4]])
            qc.cz(qr[perm[2]],qr[perm[3]])
            qc.cz(qr[perm[4]],qr[perm[5]])
            qc.cz(qr[perm[1]],qr[perm[5]])

            # Check if the state preparation is okay
            
            # Check 1
            qc.reset(anc)
            qc.h(anc[0])
            qc.cx(anc[0],qr[perm[1]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[2]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[5]])
            
            qc.h(anc[0])
            qc.measure(anc[0],reg[0])
            qc.measure(anc[1],reg[1])
            
            job = execute(qc,backend)
            result = job.result()
            data = result.get_counts()
            print(data)
            
            
            
            # Check 2
            qc.reset(anc)
            qc.h(anc[0])
            qc.cz(anc[0],qr[perm[1]])
            qc.cx(anc[0],anc[1])
            qc.cx(anc[0],qr[perm[2]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[3]])
            qc.h(anc[0])
            qc.measure(anc[0],reg[0])
            qc.measure(anc[1],reg[1])
            
            job = execute(qc,backend)
            result = job.result()
            data = result.get_counts()
            print(data)
            
            # Check 3
            qc.reset(anc)
            qc.h(anc[0])
            qc.cx(anc[0],qr[perm[4]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[3]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[5]])
            qc.h(anc[0])
            qc.measure(anc[0],reg[0])
            qc.measure(anc[1],reg[1])
            
            job = execute(qc,backend)
            result = job.result()
            data = result.get_counts()
            print(data)
            
            break
            
        
    def setInput(self, qc, compiler):
        qr = getQReg(qc,0)
        
        if compiler == 0: 
            return
        elif compiler == 1:
            qc.x(qr)
        else:
            print("Invalid compiler")
        qc.barrier()

    # Measure x from qubit a to target b
    def xMeas(self,qc,a,b):
        qr = getQReg(qc,0)
        anc = getQReg(qc,1)
        
        a = self.perm[a]
        b = self.ind[b]
        qc.h(qr[a])
        qc.cx(qr[a],anc[b])
        qc.h(qr[a])
    
    # Measure z from qubit a to target b
    def zMeas(self,qc,a,b):
        qr = getQReg(qc,0)
        anc = getQReg(qc,1)
        
        a = self.perm[a]
        b = self.ind[b]
        qc.cx(qr[a],anc[b])

    # stab is stabilizer number, flag is boolean 
    def measure(self, qc, stab, flag):
        qr = getQReg(qc,0)
        anc = getQReg(qc,1)
        reg = getCReg(qc,1)
        
        qc.reset(anc)


        # stabilizer 0 is XZXXI
        if stab == 0:
            if flag:
                qc.h(anc[0])
            #self.xMeas(qc,1,6)
            if flag:
                qc.cx(anc[0],anc[1])
            self.zMeas(qc,2,6)
            self.zMeas(qc,3,6)
            if flag:
                qc.cx(anc[0],anc[1])
            #self.xMeas(qc,4,6)

            # Check syndrome
            qc.measure(anc[1],reg[0])

            # Check flag
            if flag:
                qc.h(anc[0])
                qc.measure(anc[0],reg[1])
            
            
            job = execute(qc,backend)
            result = job.result()
            data = result.get_counts()

            print(data)
            
            # This will probably be a problem on actual quantum computer
            # Extract syndrome
            synd = max(data.items(), key=operator.itemgetter(1))[0]
            
            print("Synd: "+str(synd))
            
            if flag:            
                print("Flag: "+str(reg[1]))

In [106]:
def getReg(qc,k):
    desired = list(qc.get_qregs().items())
    return desired[k][1]

scheme = FiveOneThree()

n = scheme.n
validOutputs = scheme.validOutputs
perm = scheme.perm
scheme.setPerm()

qr = QuantumRegister(n)
cr = ClassicalRegister(n)
qc = QuantumCircuit(qr,cr)


scheme.prepareZeros(qc)


#scheme.setInput(qc,0)

#scheme.measure(qc,0,True)


{'00 00000': 1024}
{'00 00000': 1024}
{'00 00000': 1024}
